In [2]:
import pandas as pd
import numpy as np
import pmdarima as pmd
import matplotlib.pyplot as plt
import re
from collections import Counter
import math
import itertools
import datetime
import mlxtend
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
## had to change the encoding to read the csv
df=pd.read_csv('/Users/APinkerton/Downloads/OnlineRetail.csv',encoding='latin1')

## changing column titles to lower case, just because I prefer them
df.columns=df.columns.str.lower()

## stripping the product descriptions of whitespace on either side
df['description'] = df['description'].str.strip()

## dropping the NaN invoices that won't be helpful for this analysis of market baskets & looking at 
## what was bought together
df.dropna(axis=0, subset=['invoiceno'], inplace=True)

## changing the Integer to strings, as the Invoice shouldn't be treated as a number
df['invoiceno'] = df['invoiceno'].astype('str')

## Invoices with 'C' are credit, so for the time being we can filter them out--a separate analsys
## for JUST these transactions may be useful in the future
df = df[~df['invoiceno'].str.contains('C')]


In [4]:
## Grouping customer baskets by the invoice number and product description & aggregating the sales units 
## by product description, filling the NaN with 0's & setting the transaction ID as the index of the DataFrame
baskets = (df.groupby(['invoiceno', 'description'])['quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('invoiceno'))
baskets

description,*Boombox Ipod Classic,*USB Office Mirror Ball,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
invoiceno,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581586,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A563185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
## Creating a function--if a product's units are 1 or more, it counts as a sale, if it it 0 or a return,
## it doesn't count as a sale
def bool_unit(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [10]:
## Mapping the boolean unit to all of the products in the dataframe
item_sets = baskets.applymap(bool_unit)

## Dropping charges for shipping from the dataframe
item_sets.drop('POSTAGE', inplace=True, axis=1)

In [13]:
## Setting parameters for the apriori function, with a Min Support of 2.5% because there are so many products,
## and using the column names, which correspond to the product title
frequent_itemsets = apriori(item_sets, min_support=0.025, use_colnames=True)

## Inspecting 
frequent_itemsets

,support,itemsets
0,0.046337,(6 RIBBONS RUSTIC CHARM)
1,0.029985,(60 CAKE CASES VINTAGE CHRISTMAS)
2,0.040175,(60 TEATIME FAIRY CAKE CASES)
3,0.029840,(72 SWEETHEART FAIRY CAKE CASES)
4,0.047550,(ALARM CLOCK BAKELIKE GREEN)
...,...,...
201,0.026783,"(PAPER CHAIN KIT VINTAGE CHRISTMAS, PAPER CHAI..."
202,0.029064,"(ROSES REGENCY TEACUP AND SAUCER, PINK REGENCY..."
203,0.025473,"(ROSES REGENCY TEACUP AND SAUCER, REGENCY CAKE..."
204,0.026152,"(WOODEN FRAME ANTIQUE WHITE, WOODEN PICTURE FR..."


In [18]:
## Creating a rule for the association rules--Using lift as the metric and a minimum threshold of 1
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

## Filtering the rules dataframe to items with association lift greater than 500% and with a confidence of >50%
## Which yields 61 product association rules
rules[(rules['lift']>5)&(rules['conviction']>.5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.047550,0.050995,0.031053,0.653061,12.806462,0.028628,2.735368
1,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.050995,0.047550,0.031053,0.608944,12.806462,0.028628,2.435585
2,(RED RETROSPOT CHARLOTTE BAG),(CHARLOTTE BAG PINK POLKADOT),0.050170,0.036050,0.025328,0.504836,14.003582,0.023519,1.946726
3,(CHARLOTTE BAG PINK POLKADOT),(RED RETROSPOT CHARLOTTE BAG),0.036050,0.050170,0.025328,0.702557,14.003582,0.023519,3.193320
4,(SPACEBOY LUNCH BOX),(DOLLY GIRL LUNCH BOX),0.043668,0.041242,0.026104,0.597778,14.494353,0.024303,2.383652
5,(DOLLY GIRL LUNCH BOX),(SPACEBOY LUNCH BOX),0.041242,0.043668,0.026104,0.632941,14.494353,0.024303,2.605391
6,(GARDENERS KNEELING PAD CUP OF TEA),(GARDENERS KNEELING PAD KEEP CALM),0.036778,0.044299,0.026492,0.720317,16.260379,0.024863,3.417082
7,(GARDENERS KNEELING PAD KEEP CALM),(GARDENERS KNEELING PAD CUP OF TEA),0.044299,0.036778,0.026492,0.598028,16.260379,0.024863,2.396244
8,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.049248,0.037166,0.030713,0.623645,16.779804,0.028883,2.558314
9,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037166,0.049248,0.030713,0.826371,16.779804,0.028883,5.475760
